In [1]:
# Define the packages
using Pkg
using CategoricalArrays
using MAT
using DataFrames
using MLJ
using LinearAlgebra
using Statistics
using Plots
using StatsPlots
using CSV

In [3]:
# sub-MOA101, sub-MOA102, sub-MOA104, sub-MOA105, sub-MOA107, sub-MOA108, sub-MOA109, sub-MOA110, sub-MOA111, sub-MOA112, sub-MOA114,
# sub-MOA115, sub-MOA116, sub-MOA118, sub-MOA121, sub-MOA122, sub-MOA123, sub-MOA124, sub-MOA126, sub-MOA127, sub-MOA128, sub-MOA130,
# sub-MOA131, sub-MOA133, sub-MOA134, sub-MOA135

# Binary treatment response categories
y_binary = ("Response","Response","Response","No Response","Response","No Response","Response","No Response","Response","Response",
    "No Response","No Response","Response","Response","No Response","No Response","No Response","No Response","Response","Response",
    "No Response", "No Response" ,"No Response", "No Response","Response", "No Response")

# Tri label response categories
y_tri = ("Partial Response","Partial Response","Remission","No Response","Partial Response","No Response","Partial Response","No Response",
    "Remission", "Remission","Partial Response","Partial Response","Partial Response","Remission","No Response","No Response",
    "Partial Response","No Response","Remission","Partial Response","Partial Response","Partial Response","No Response","No Response",
    "Remission","No Response")

# Quad label response categories
y_quad = ("Partial Response","Partial Response","Remission","Stable","Partial Response","Deterioration","Partial Response","Stable",
    "Remission","Remission","Partial Response","Partial Response","Partial Response","Remission","Stable","Deterioration", "Stable",
    "Stable","Remission","Partial Response","Partial Response","Partial Response","Stable","Stable","Remission","Stable")

# Penta label response categories
y_penta = ("Strong Response","Strong Response","Remission","Stable","Strong Response","Deterioration","Strong Response","Stable",
    "Remission","Remission","Mild Response","Mild Response","Strong Response","Remission","Stable","Deterioration","Stable","Stable",
    "Remission","Strong Response","Mild Response","Mild Response","Stable","Stable","Remission","Stable")



# Convert to CategoricalArray
y_binary = categorical(collect(y_binary))
y_tri = categorical(collect(y_tri))
y_quad = categorical(collect(y_quad))
y_penta = categorical(collect(y_penta))


26-element CategoricalArray{String,1,UInt32}:
 "Strong Response"
 "Strong Response"
 "Remission"
 "Stable"
 "Strong Response"
 "Deterioration"
 "Strong Response"
 "Stable"
 "Remission"
 "Remission"
 ⋮
 "Stable"
 "Remission"
 "Strong Response"
 "Mild Response"
 "Mild Response"
 "Stable"
 "Stable"
 "Remission"
 "Stable"

In [4]:
Pkg.add("ProgressMeter")
Pkg.instantiate()

   Resolving package versions...
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Project.toml`
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Manifest.toml`


In [5]:
using Random, Statistics, ProgressMeter

"""
    permutation_test_classifier(
        X::Tables.AbstractColumns, y::AbstractVector,
        model; cv=nothing, n_perm=1000, measure=accuracy, rng=Random.default_rng()
    ) → (p, observed, null_dist)

Exact-label permutation test for classification metrics.

* `model`     - an MLJ model instance (e.g. LogisticClassifier()).
* `cv`        - a pre-built `StratifiedCV` or `CV`; if `nothing` a 5-fold stratified CV is built.
* `n_perm`    - number of label shuffles.
* `measure`   - any MLJ measure (default = `accuracy`).

Returns the *p*-value, the observed score, and the vector of permuted scores.
"""
function permutation_test_classifier(X, y, model;
        cv=nothing, n_perm::Int=1000,
        measure=accuracy, rng=Random.default_rng())

    cv === nothing && (cv = StratifiedCV(nfolds=5, shuffle=true, rng=rng))
    obs_mach = machine(model, X, y)
    obs_result = evaluate!(obs_mach; resampling=cv, measures=[measure],
                           operation=predict_mode, verbosity=0)
    observed = mean(obs_result.per_fold[1])

    null_scores = Float64[]
    @showprogress 1 "Permutation test…" for _ in 1:n_perm
        y_perm = shuffle(rng, y)
        perm_mach = machine(model, X, y_perm)
        res = evaluate!(perm_mach; resampling=cv, measures=[measure],
                        operation=predict_mode, verbosity=0)
        push!(null_scores, mean(res.per_fold[1]))
    end
    # one-sided p-value (H_0: score is not better than chance)
    p = (count(>=(observed), null_scores) + 1) / (n_perm + 1)

    return p, observed, null_scores
end



permutation_test_classifier

In [6]:
# List of subjects that have some degree of depression
target_subjects = [
    "sub-MOA101", "sub-MOA102", "sub-MOA104", "sub-MOA105", "sub-MOA107", "sub-MOA108", "sub-MOA109", "sub-MOA110", "sub-MOA111",
    "sub-MOA112", "sub-MOA114", "sub-MOA115", "sub-MOA116", "sub-MOA118", "sub-MOA121", "sub-MOA122", "sub-MOA123","sub-MOA124", 
    "sub-MOA126", "sub-MOA127", "sub-MOA128", "sub-MOA130", "sub-MOA131", "sub-MOA133", "sub-MOA134", "sub-MOA135"]

# Base path to your subject folders
base_path = "Spectral_DCM_Collection_15x15"

# Collect valid file paths
valid_files = String[]

for subj in target_subjects
    subj_path = joinpath(base_path, subj)
    ses_path = joinpath(subj_path, "ses-b0")
    glm_path = joinpath(ses_path, "glm")
    dcm_file = joinpath(glm_path, "spDCM_rsTozzi.mat")

    if !isdir(ses_path)
        @warn "Missing session folder: $ses_path"
    elseif !isfile(dcm_file)
        @warn "Missing spDCM_rsTozzi.mat for $subj"
    else
        push!(valid_files, dcm_file)
    end
end

println("✅ Found spDCM_rsTozzi.mat for $(length(valid_files)) out of $(length(target_subjects)) subjects.")


✅ Found spDCM_rsTozzi.mat for 26 out of 26 subjects.


In [7]:
# Extract A matrix features as a flat 16-element vector
function extract_features(file)
    mat = matread(file)
    A = mat["params"]  # 4×4 matrix
    return vec(Matrix(A))  # Flatten to 16-element vector
end

extract_features (generic function with 1 method)

In [8]:
# Create feature dataset
X = hcat([extract_features(file) for file in valid_files]...)'

X_df = DataFrame(X, :auto)  # convert to MLJ-compatible table

# Ensure that the number of subjects match in X_df and Y labels of different modalities
@assert size(X_df, 1) == length(y_binary) "Mismatch between number of samples in X and y_binary"
@assert size(X_df, 1) == length(y_tri) "Mismatch between number of samples in X and y_tri"
@assert size(X_df, 1) == length(y_quad) "Mismatch between number of samples in X and y_quad"
@assert size(X_df, 1) == length(y_penta) "Mismatch between number of samples in X and y_penta"

In [9]:
function evaluate_logistic_model(X_df, y, nfolds::Int, label_type::String)
    # Load model
    LogisticClassifier = @load LogisticClassifier pkg=MLJLinearModels
    pipe = LogisticClassifier()
    mach = machine(pipe, X_df, y)

    # Setup evaluation
    measures = [accuracy, MulticlassFScore(), ConfusionMatrix()]
    cv = StratifiedCV(nfolds=nfolds, shuffle=true, rng=42)

    # Evaluate
    eval_result = evaluate!(
        mach,
        resampling=cv,
        measures=measures,
        operation=predict_mode,
        verbosity=0,
        check_measure=false
    )

    accs = eval_result.per_fold[1]
    f1s = eval_result.per_fold[2]
    cmats = eval_result.per_fold[3]

    macro_precisions = Float64[]
    macro_recalls = Float64[]

    for cm in cmats
        cmatrix = cm.mat
        TP = diag(cmatrix)
        FP = sum(cmatrix, dims=1)' .- TP
        FN = sum(cmatrix, dims=2) .- TP
        support = sum(cmatrix, dims=2)[:]
        total = sum(support)

        class_precisions = map((tp, fp) -> (tp + fp == 0 ? NaN : tp / (tp + fp)), TP, FP)
        class_recalls    = map((tp, fn) -> (tp + fn == 0 ? NaN : tp / (tp + fn)), TP, FN)

        macro_precision = mean(skipmissing(class_precisions))
        macro_recall    = mean(skipmissing(class_recalls))

        push!(macro_precisions, macro_precision)
        push!(macro_recalls, macro_recall)
    end

    # Collect metrics in DataFrame
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        Accuracy = accs,
        F1_Score = f1s,
        Macro_Precision = macro_precisions,
        Macro_Recall = macro_recalls
    )

    # Add row for averages
    avg_row = DataFrame(
        Fold = ["Average"],
        Accuracy = [mean(skipmissing(accs))],
        F1_Score = [mean(skipmissing(f1s))],
        Macro_Precision = [mean(skipmissing(macro_precisions))],
        Macro_Recall = [mean(skipmissing(macro_recalls))]
    )

    metrics_table = vcat(metrics_df, avg_row)

    # Save table as CSV instead (more robust and universally usable)
    csv_table_path = "tables/table_$(nfolds)_grouped_metrics_logistics_$(label_type)_15x15.csv"
    CSV.write(csv_table_path, metrics_table)

    # Prepare DataFrame for plotting
    metrics = ["Accuracy", "F1 Score", "Macro Precision", "Macro Recall"]
    metric_values = [accs, f1s, macro_precisions, macro_recalls]

    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metrics, inner=nfolds),
        Value = vcat(metric_values...)
    )

    # Plot grouped bars
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        yticks=0:0.05:1.0,
        title="Logistic Classification Fold-wise Metrics",
        legend=:outertop,
        size=(750, 500),
        guidefontsize=10,
        tickfontsize=10,
        dpi=300
    )

    savefig("figures/$(nfolds)_grouped_metrics_logistics_$(label_type)_15x15.png")
end

evaluate_logistic_model (generic function with 1 method)

In [10]:
# #Evaluate logistic model classifier for 3 fold cross validation
# evaluate_logistic_model(X_df, y_binary, 3, "binary")
# evaluate_logistic_model(X_df, y_tri, 3, "tri")
# evaluate_logistic_model(X_df, y_quad, 3, "quad")
# evaluate_logistic_model(X_df, y_penta, 3, "penta")

# #Evaluate logistic model classifier for 4 fold cross validation
# evaluate_logistic_model(X_df, y_binary, 4, "binary")
# evaluate_logistic_model(X_df, y_tri, 4, "tri")
# evaluate_logistic_model(X_df, y_quad, 4, "quad")
# evaluate_logistic_model(X_df, y_penta, 4, "penta")

# #Evaluate logistic model classifier for 5 fold cross validation
# evaluate_logistic_model(X_df, y_binary, 5, "binary")
# evaluate_logistic_model(X_df, y_tri, 5, "tri")
# evaluate_logistic_model(X_df, y_quad, 5, "quad")
# evaluate_logistic_model(X_df, y_penta, 5, "penta")

In [11]:
using DataFrames
using CSV
using MLJ
using MLJLinearModels

"""
    evaluate_and_save_logistic_models(X_df, y_binary, y_tri, y_quad, y_penta; n_perm=500, outpath="evaluation_results.csv")

Runs 'evaluate_logistic_model' for 3-, 4-, and 5-fold cross-validation on each of
the targets (binary, tri, quad, penta), then runs a permutation test for each target
(using 'permutation_test_classifier' with 'n_perm' permutations). Collects all results
into a single DataFrame and writes it out to `outpath` in CSV format.

# Arguments
- 'X_df' : feature table (DataFrame or MLJ‐compatible table).
- 'y_binary', 'y_tri', 'y_quad', 'y_penta' : categorical target vectors for each label type.
- 'n_perm' : number of permutations for each permutation test (default: 500).
- 'outpath' : file path for saving the results CSV (default: "evaluation_results.csv").

# Returns
A 'DataFrame' with columns:
- :fold (Int) — the number of CV folds used (3, 4, or 5).
- :target (String) — one of "binary", "tri", "quad", "penta".
- :eval_result (Any) — whatever 'evaluate_logistic_model' returned (usually a DataFrame of per-fold metrics plus average).
- :perm_p_value (Float64) — p-value from the permutation test.
- :perm_observed_accuracy (Float64) — observed accuracy from the permutation test.
- :perm_null_mean_accuracy (Float64) — mean of the null accuracies (mean(null_scores)).

The CSV will be saved to outpath.
"""
function evaluate_and_save_logistic_models(
    X_df,
    y_binary,
    y_tri,
    y_quad,
    y_penta;
    n_perm::Int = 500,
    outpath::AbstractString = "evaluation_results.csv"
)
    # Load the logistic classifier once
    LogisticClassifier = @load LogisticClassifier pkg = MLJLinearModels

    folds = [3, 4, 5]
    y_list = [y_binary, y_tri, y_quad, y_penta]
    labels = ["binary", "tri", "quad", "penta"]

    # Prepare an empty DataFrame to collect all results
    results = DataFrame(
        fold                     = Int[],
        target                   = String[],
        eval_result              = Any[],
        perm_p_value             = Float64[],
        perm_observed_accuracy   = Float64[],
        perm_null_mean_accuracy  = Float64[],
    )

    for fold in folds
        for (y, label) in zip(y_list, labels)
            # 1) Run the k‐fold evaluation
            eval_res = evaluate_logistic_model(X_df, y, fold, label)

            # 2) Run the permutation test on this same target
            p_val, acc_obs, acc_null = permutation_test_classifier(
                X_df,
                y,
                LogisticClassifier(); 
                n_perm = n_perm
            )

            null_mean = mean(acc_null)

            # 3) Append a row to the DataFrame
            push!(
                results,
                (
                    fold,
                    label,
                    eval_res,
                    p_val,
                    acc_obs,
                    null_mean,
                )
            )
        end
    end

    # 4) Write to CSV
    CSV.write(outpath, results)

    return results
end


evaluate_and_save_logistic_models

In [12]:
df = evaluate_and_save_logistic_models(
    X_df, y_binary, y_tri, y_quad, y_penta;
    n_perm = 500,
    outpath = "15x15_evaluation_results_logistic.csv"
)


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔


Permutation test… 100%|██████████████████████████████████| Time: 0:00:01
┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


...

import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:02


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:03


Row,fold,target,eval_result,perm_p_value,perm_observed_accuracy,perm_null_mean_accuracy
,Int64,String,Any,Float64,Float64,Float64
1,3,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_logistics_binary_15x15.png,0.391218,0.54,0.501933
2,3,tri,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_logistics_tri_15x15.png,0.0798403,0.54,0.367333
3,3,quad,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_logistics_quad_15x15.png,0.0618762,0.466667,0.309013
4,3,penta,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_logistics_penta_15x15.png,0.0159681,0.466667,0.23492
5,4,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_logistics_binary_15x15.png,0.409182,0.533333,0.502813
6,4,tri,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_logistics_tri_15x15.png,0.0658683,0.513333,0.354373
7,4,quad,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_logistics_quad_15x15.png,0.461078,0.313333,0.308467
8,4,penta,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_logistics_penta_15x15.png,0.205589,0.313333,0.236667
9,5,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/5_grouped_metrics_logistics_binary_15x15.png,0.0818363,0.66,0.494267


In [16]:
function evaluate_multinomial_classifier_model(X_df, y, nfolds::Int, label_type::String)
    # Load model
    MultinomialClassifier = @load MultinomialClassifier pkg=MLJLinearModels
    pipe = MultinomialClassifier()
    mach = machine(pipe, X_df, y)

    # Setup evaluation
    measures = [accuracy, MulticlassFScore(), ConfusionMatrix()]
    cv = StratifiedCV(nfolds=nfolds, shuffle=true, rng=42)

    # Evaluate
    eval_result = evaluate!(
        mach,
        resampling=cv,
        measures=measures,
        operation=predict_mode,
        verbosity=0,
        check_measure=false
    )

    accs = eval_result.per_fold[1]
    f1s = eval_result.per_fold[2]
    cmats = eval_result.per_fold[3]

    macro_precisions = Float64[]
    macro_recalls = Float64[]

    for cm in cmats
        cmatrix = cm.mat
        TP = diag(cmatrix)
        FP = sum(cmatrix, dims=1)' .- TP
        FN = sum(cmatrix, dims=2) .- TP
        support = sum(cmatrix, dims=2)[:]
        total = sum(support)

        class_precisions = map((tp, fp) -> (tp + fp == 0 ? NaN : tp / (tp + fp)), TP, FP)
        class_recalls    = map((tp, fn) -> (tp + fn == 0 ? NaN : tp / (tp + fn)), TP, FN)

        macro_precision = mean(skipmissing(class_precisions))
        macro_recall    = mean(skipmissing(class_recalls))

        push!(macro_precisions, macro_precision)
        push!(macro_recalls, macro_recall)
    end

    # Collect metrics in DataFrame
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        Accuracy = accs,
        F1_Score = f1s,
        Macro_Precision = macro_precisions,
        Macro_Recall = macro_recalls
    )

    # Add row for averages
    avg_row = DataFrame(
        Fold = ["Average"],
        Accuracy = [mean(skipmissing(accs))],
        F1_Score = [mean(skipmissing(f1s))],
        Macro_Precision = [mean(skipmissing(macro_precisions))],
        Macro_Recall = [mean(skipmissing(macro_recalls))]
    )

    metrics_table = vcat(metrics_df, avg_row)

    # Save table as CSV instead (more robust and universally usable)
    csv_table_path = "tables/table_$(nfolds)_grouped_metrics_multinomial_$(label_type)_15x15.csv"
    CSV.write(csv_table_path, metrics_table)

    # Prepare DataFrame for plotting
    metrics = ["Accuracy", "F1 Score", "Macro Precision", "Macro Recall"]
    metric_values = [accs, f1s, macro_precisions, macro_recalls]

    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metrics, inner=nfolds),
        Value = vcat(metric_values...)
    )

    # Plot grouped bars
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        yticks=0:0.05:1.0,
        title="Multinomial Classification Fold-wise Metrics",
        legend=:outertop,
        size=(750, 500),
        guidefontsize=10,
        tickfontsize=10,
        dpi=300
    )

    savefig("figures/$(nfolds)_grouped_metrics_multinomial_$(label_type)_15x15.png")
end

evaluate_multinomial_classifier_model (generic function with 1 method)

In [17]:
# Permutation test saver for multinomial classifiers

function evaluate_and_save_multinomial(
    X_df,
    y_binary,
    y_tri,
    y_quad,
    y_penta;
    n_perm::Int = 500,
    outpath::AbstractString = "evaluation_results.csv"
)
    # Load the logistic classifier once
    LogisticClassifier = @load LogisticClassifier pkg = MLJLinearModels

    folds = [3, 4, 5]
    y_list = [y_binary, y_tri, y_quad, y_penta]
    labels = ["binary", "tri", "quad", "penta"]

    # Prepare an empty DataFrame to collect all results
    results = DataFrame(
        fold                     = Int[],
        target                   = String[],
        eval_result              = Any[],
        perm_p_value             = Float64[],
        perm_observed_accuracy   = Float64[],
        perm_null_mean_accuracy  = Float64[],
    )

    for fold in folds
        for (y, label) in zip(y_list, labels)
            # 1) Run the k‐fold evaluation
            eval_res = evaluate_multinomial_classifier_model(X_df, y, fold, label)

            # 2) Run the permutation test on this same target
            p_val, acc_obs, acc_null = permutation_test_classifier(
                X_df,
                y,
                LogisticClassifier(); 
                n_perm = n_perm
            )

            null_mean = mean(acc_null)

            # 3) Append a row to the DataFrame
            push!(
                results,
                (
                    fold,
                    label,
                    eval_res,
                    p_val,
                    acc_obs,
                    null_mean,
                )
            )
        end
    end

    # 4) Write to CSV
    CSV.write(outpath, results)

    return results
end


evaluate_and_save_multinomial (generic function with 1 method)

In [18]:
df = evaluate_and_save_multinomial(
    X_df, y_binary, y_tri, y_quad, y_penta;
    n_perm = 500,
    outpath = "15x15_evaluation_results_multinomial.csv"
)

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔
import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:01


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:02


...

import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:03


import MLJLinearModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:03


Row,fold,target,eval_result,perm_p_value,perm_observed_accuracy,perm_null_mean_accuracy
,Int64,String,Any,Float64,Float64,Float64
1,3,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_multinomial_binary_15x15.png,0.532934,0.493333,0.502067
2,3,tri,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_multinomial_tri_15x15.png,0.0319361,0.566667,0.34828
3,3,quad,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_multinomial_quad_15x15.png,0.305389,0.353333,0.307587
4,3,penta,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_multinomial_penta_15x15.png,0.203593,0.32,0.24012
5,4,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_multinomial_binary_15x15.png,0.241517,0.606667,0.51424
6,4,tri,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_multinomial_tri_15x15.png,0.0598802,0.513333,0.35116
7,4,quad,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_multinomial_quad_15x15.png,0.676647,0.266667,0.310893
8,4,penta,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_multinomial_penta_15x15.png,0.566866,0.22,0.237427
9,5,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/5_grouped_metrics_multinomial_binary_15x15.png,0.153693,0.62,0.4926


In [14]:
# #Evaluate multinomial model classifier for 3 fold cross validation
# evaluate_multinomial_classifier_model(X_df, y_binary, 3, "binary")
# evaluate_multinomial_classifier_model(X_df, y_tri, 3, "tri")
# evaluate_multinomial_classifier_model(X_df, y_quad, 3, "quad")
# evaluate_multinomial_classifier_model(X_df, y_penta, 3, "penta")

# #Evaluate multinomial model classifier for 4 fold cross validation
# evaluate_multinomial_classifier_model(X_df, y_binary, 4, "binary")
# evaluate_multinomial_classifier_model(X_df, y_tri, 4, "tri")
# evaluate_multinomial_classifier_model(X_df, y_quad, 4, "quad")
# evaluate_multinomial_classifier_model(X_df, y_penta, 4, "penta")

# #Evaluate multinomial model classifier for 5 fold cross validation
# evaluate_multinomial_classifier_model(X_df, y_binary, 5, "binary")
# evaluate_multinomial_classifier_model(X_df, y_tri, 5, "tri")
# evaluate_multinomial_classifier_model(X_df, y_quad, 5, "quad")
# evaluate_multinomial_classifier_model(X_df, y_penta, 5, "penta")

In [19]:
Pkg.add("MLJScikitLearnInterface")
Pkg.instantiate()

   Resolving package versions...
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Project.toml`
  No Changes to `~/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/Manifest.toml`


In [20]:
using MLJScikitLearnInterface
function evaluate_random_forest_model(X_df, y, nfolds::Int, label_type::String)
    # Load model
    RandomForestClassifier = @load RandomForestClassifier pkg=MLJScikitLearnInterface
    pipe = RandomForestClassifier()
    mach = machine(pipe, X_df, y)

    # Setup evaluation
    measures = [accuracy, MulticlassFScore(), ConfusionMatrix()]
    cv = StratifiedCV(nfolds=nfolds, shuffle=true, rng=42)

    # Evaluate
    eval_result = evaluate!(
        mach,
        resampling=cv,
        measures=measures,
        operation=predict_mode,
        verbosity=0,
        check_measure=false
    )

    accs = eval_result.per_fold[1]
    f1s = eval_result.per_fold[2]
    cmats = eval_result.per_fold[3]

    macro_precisions = Float64[]
    macro_recalls = Float64[]

    for cm in cmats
        cmatrix = cm.mat
        TP = diag(cmatrix)
        FP = sum(cmatrix, dims=1)' .- TP
        FN = sum(cmatrix, dims=2) .- TP
        support = sum(cmatrix, dims=2)[:]
        total = sum(support)

        class_precisions = map((tp, fp) -> (tp + fp == 0 ? NaN : tp / (tp + fp)), TP, FP)
        class_recalls    = map((tp, fn) -> (tp + fn == 0 ? NaN : tp / (tp + fn)), TP, FN)

        macro_precision = mean(skipmissing(class_precisions))
        macro_recall    = mean(skipmissing(class_recalls))

        push!(macro_precisions, macro_precision)
        push!(macro_recalls, macro_recall)
    end

    # Collect metrics in DataFrame
    metrics_df = DataFrame(
        Fold = 1:nfolds,
        Accuracy = accs,
        F1_Score = f1s,
        Macro_Precision = macro_precisions,
        Macro_Recall = macro_recalls
    )

    # Add row for averages
    avg_row = DataFrame(
        Fold = ["Average"],
        Accuracy = [mean(skipmissing(accs))],
        F1_Score = [mean(skipmissing(f1s))],
        Macro_Precision = [mean(skipmissing(macro_precisions))],
        Macro_Recall = [mean(skipmissing(macro_recalls))]
    )

    metrics_table = vcat(metrics_df, avg_row)

    # Save table as CSV instead (more robust and universally usable)
    csv_table_path = "tables/table_$(nfolds)_grouped_metrics_random_forest_$(label_type)_15x15.csv"
    CSV.write(csv_table_path, metrics_table)

    # Prepare DataFrame for plotting
    metrics = ["Accuracy", "F1 Score", "Macro Precision", "Macro Recall"]
    metric_values = [accs, f1s, macro_precisions, macro_recalls]

    plot_df = DataFrame(
        Fold = repeat(1:nfolds, outer=length(metrics)),
        Metric = repeat(metrics, inner=nfolds),
        Value = vcat(metric_values...)
    )

    # Plot grouped bars
    @df plot_df groupedbar(
        string.(:Fold), :Value, group=:Metric,
        bar_position=:dodge,
        bar_width=0.2,
        xlabel="Fold", ylabel="Metric Value",
        yticks=0:0.05:1.0,
        title="Random Forest Classification Fold-wise Metrics",
        legend=:outertop,
        size=(750, 500),
        guidefontsize=10,
        tickfontsize=10,
        dpi=300
    )

    savefig("figures/$(nfolds)_grouped_metrics_random_forest_$(label_type)_15x15.png")
end

evaluate_random_forest_model (generic function with 1 method)

In [22]:
# Permutation test saver for random forest classifiers

function evaluate_and_save_random_forest(
    X_df,
    y_binary,
    y_tri,
    y_quad,
    y_penta;
    n_perm::Int = 500,
    outpath::AbstractString = "evaluation_results.csv"
)
    # Load the logistic classifier once
    LogisticClassifier = @load LogisticClassifier pkg = MLJLinearModels

    folds = [3, 4, 5]
    y_list = [y_binary, y_tri, y_quad, y_penta]
    labels = ["binary", "tri", "quad", "penta"]

    # Prepare an empty DataFrame to collect all results
    results = DataFrame(
        fold                     = Int[],
        target                   = String[],
        eval_result              = Any[],
        perm_p_value             = Float64[],
        perm_observed_accuracy   = Float64[],
        perm_null_mean_accuracy  = Float64[],
    )

    for fold in folds
        for (y, label) in zip(y_list, labels)
            # 1) Run the k‐fold evaluation
            eval_res = evaluate_random_forest_model(X_df, y, fold, label)

            # 2) Run the permutation test on this same target
            p_val, acc_obs, acc_null = permutation_test_classifier(
                X_df,
                y,
                LogisticClassifier(); 
                n_perm = n_perm
            )

            null_mean = mean(acc_null)

            # 3) Append a row to the DataFrame
            push!(
                results,
                (
                    fold,
                    label,
                    eval_res,
                    p_val,
                    acc_obs,
                    null_mean,
                )
            )
        end
    end

    # 4) Write to CSV
    CSV.write(outpath, results)

    return results
end


evaluate_and_save_random_forest (generic function with 1 method)

In [23]:
df = evaluate_and_save_random_forest(
    X_df, y_binary, y_tri, y_quad, y_penta;
    n_perm = 500,
    outpath = "15x15_evaluation_results_random_forest.csv"
)

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159


import MLJLinearModels ✔
import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:01


import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:02


...

import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:02


import MLJScikitLearnInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/ricobenning/.julia/packages/MLJModels/nxeCf/src/loading.jl:159
Permutation test… 100%|██████████████████████████████████| Time: 0:00:03


Row,fold,target,eval_result,perm_p_value,perm_observed_accuracy,perm_null_mean_accuracy
,Int64,String,Any,Float64,Float64,Float64
1,3,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_random_forest_binary_15x15.png,0.125749,0.646667,0.50324
2,3,tri,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_random_forest_tri_15x15.png,0.0538922,0.54,0.358853
3,3,quad,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_random_forest_quad_15x15.png,0.43513,0.34,0.312227
4,3,penta,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/3_grouped_metrics_random_forest_penta_15x15.png,0.141717,0.346667,0.240467
5,4,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_random_forest_binary_15x15.png,0.518962,0.506667,0.507133
6,4,tri,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_random_forest_tri_15x15.png,0.357285,0.386667,0.353933
7,4,quad,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_random_forest_quad_15x15.png,0.273453,0.386667,0.313653
8,4,penta,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/4_grouped_metrics_random_forest_penta_15x15.png,0.0179641,0.453333,0.23268
9,5,binary,/Users/ricobenning/Python_C_ML/Translational Neuromodeling/Project/Project_8/Classification_and_Regression_DCM/figures/5_grouped_metrics_random_forest_binary_15x15.png,0.55489,0.493333,0.499813


In [21]:
# #Evaluate random forest model classifier for 3 fold cross validation
# evaluate_random_forest_model(X_df, y_binary, 3, "binary")
# evaluate_random_forest_model(X_df, y_tri, 3, "tri")
# evaluate_random_forest_model(X_df, y_quad, 3, "quad")
# evaluate_random_forest_model(X_df, y_penta, 3, "penta")

# #Evaluate random forest model classifier for 4 fold cross validation
# evaluate_random_forest_model(X_df, y_binary, 4, "binary")
# evaluate_random_forest_model(X_df, y_tri, 4, "tri")
# evaluate_random_forest_model(X_df, y_quad, 4, "quad")
# evaluate_random_forest_model(X_df, y_penta, 4, "penta")

# #Evaluate random forest model classifier for 5 fold cross validation
# evaluate_random_forest_model(X_df, y_binary, 5, "binary")
# evaluate_random_forest_model(X_df, y_tri, 5, "tri")
# evaluate_random_forest_model(X_df, y_quad, 5, "quad")
# evaluate_random_forest_model(X_df, y_penta, 5, "penta")